In [1]:
import csv
import random
import math

In [2]:
def loadcsv(filename):
    lines=csv.reader(open(filename, "r"))
    dataset=list(lines)
    for i in range(len(dataset)):
        dataset[i]=[float (x) for x in dataset[i]]
    return dataset

In [3]:
def splitDataset(dataset, splitRatio):
    trainSize=int(len(dataset) * splitRatio)
    trainSet=[]
    copy=list(dataset)
    while len(trainSet)<trainSize:
        index=random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]


In [4]:
def separateByClass(dataset):
    separated= {}
    for i in range(len(dataset)):
        vector=dataset[i]
        if(vector[-1] not in separated):
            separated[vector[-1]]=[]
        separated[vector[-1]].append(vector)
    return separated

In [5]:
def mean(numbers):
    return sum(numbers)/float(len(numbers))
def stdev(numbers):
    avg=mean(numbers)
    variance=sum([pow(x-avg,2)for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

In [6]:
def summarize(dataset):
    summaries=[(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries

In [7]:
def summarizeByClass(dataset):
    separated=separateByClass(dataset)
    summaries={}
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
    return summaries

In [8]:
def calculateProbability(x, mean, stdev):
    exponent=math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return(1 / (math.sqrt(2*math.pi) * stdev)) * exponent


In [9]:
def calculateClassProbabilities(summaries, inputVector):
    probabilities={}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue]=1
        for i in range(len(classSummaries)):
            mean, stdev=classSummaries[i]
            x=inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
    return probabilities
            

In [10]:
def predict(summaries, inputVector):
    probabilities=calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb=None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb=probability
            bestLabel=classValue
    return bestLabel

In [14]:
def getPredictions(summaries, testSet):
    predictions=[]
    for i in range(len(testSet)):
            result=predict(summaries, testSet[i])
            predictions.append(result)
    return predictions

    

In [15]:
def getAccuracy(testSet, predictions):
    correct=0
    for i in range(len(testSet)):
        if testSet[i][-1]==predictions[i]:
            correct+=1
    return (correct/float(len(testSet))) * 100.0

In [16]:
def main():
    filename='data5.csv'
    splitRatio=0.67
    dataset=loadcsv(filename)
    
    print("\n the length of the Data Set:",len(dataset))
    print("\n the Data Set Splitting into Training and Testing\n")
    trainingSet, testSet=splitDataset(dataset, splitRatio)
    
    print('\n Number of Rows in Training Set:{0} rows'.format(len(trainingSet)))
    print('\n Number of Rows in Testing Set:{0} rows'.format(len(testSet)))
    print("\n First Five Rows of Training Set:\n")
    for i in range(0,5):
        print(trainingSet[i],"\n")
    print("\n First Five Rows of Testing Set:\n")
    for i in range(0,5):
        print(testSet[i],"\n")
    
    summaries=summarizeByClass(trainingSet)
    print("\n Model Summaries:\n",summaries)
   
    predictions=getPredictions(summaries, testSet)
    print("\n predictions:\n",predictions)
    
    accuracy=getAccuracy(testSet, predictions)
    print('\n Accuracy: {0}%'.format(accuracy))
main()
    


 the length of the Data Set: 768

 the Data Set Splitting into Training and Testing


 Number of Rows in Training Set:514 rows

 Number of Rows in Testing Set:254 rows

 First Five Rows of Training Set:

[2.0, 130.0, 96.0, 0.0, 0.0, 22.6, 0.268, 21.0, 0.0] 

[3.0, 100.0, 68.0, 23.0, 81.0, 31.6, 0.949, 28.0, 0.0] 

[0.0, 129.0, 80.0, 0.0, 0.0, 31.2, 0.703, 29.0, 0.0] 

[1.0, 128.0, 82.0, 17.0, 183.0, 27.5, 0.115, 22.0, 0.0] 

[13.0, 152.0, 90.0, 33.0, 29.0, 26.8, 0.731, 43.0, 1.0] 


 First Five Rows of Testing Set:

[6.0, 148.0, 72.0, 35.0, 0.0, 33.6, 0.627, 50.0, 1.0] 

[8.0, 183.0, 64.0, 0.0, 0.0, 23.3, 0.672, 32.0, 1.0] 

[5.0, 116.0, 74.0, 0.0, 0.0, 25.6, 0.201, 30.0, 0.0] 

[2.0, 197.0, 70.0, 45.0, 543.0, 30.5, 0.158, 53.0, 1.0] 

[10.0, 168.0, 74.0, 0.0, 0.0, 38.0, 0.537, 34.0, 1.0] 


 Model Summaries:
 {0.0: [(3.31044776119403, 2.97781891449638), (110.54626865671642, 24.97034365768116), (69.25074626865671, 16.21108304565961), (20.065671641791045, 14.848592669445658), (67.57014